In [1]:
import pandas as pd
from geopy.distance import geodesic
import osmnx as ox
import geopandas as gpd
from geopy.geocoders import Nominatim
from shapely.geometry import Point
import matplotlib.pyplot as plt

In [2]:
df_ContainerActivities_cleaned_coordinates = pd.read_csv('/Users/annele/PycharmProjects/Glas-O-Mat/data/modified_data/ContainerActivities_populations_density.csv')
df_ContainerActivities_cleaned_coordinates.reset_index(drop=True, inplace=True)
df_ContainerActivities_cleaned_coordinates.drop('Unnamed: 0', axis=1, inplace=True)

df_ContainerActivities_cleaned_coordinates

/var/folders/hp/y7wgcxwd5s7ff9cjh270v5v40000gp/T/ipykernel_11683/290288042.py:1: DtypeWarning: Columns (20,21) have mixed types. Specify dtype option on import or set low_memory=False.
  df_ContainerActivities_cleaned_coordinates = pd.read_csv('/Users/annele/PycharmProjects/Glas-O-Mat/data/modified_data/ContainerActivities_populations_density.csv')


,TRANSACTION_ID,CONTAINER_ID,SLIDER_LEVEL,CV_LEVEL,SENSOR_LEVEL,IMAGE_ID,PHONE_ID,RECORDED_AT,COMMENT,IS_EMPTIED,...,GEO_LON,DATE,CLASSIFICATION,outlier,DISTANCE,NEW_LAT,NEW_LON,NEW_DISTANCE,INACCURATE_COORDINATES,POPULATION_DENSITY
0,1,1001101101100271111,30,NaN,NaN,1001101101100271111/2023-09-28T06:36:48.022095...,TP1A.220624.014_2,2023-09-26 06:36:48.0,NaN,1,...,7.791667,2023-09-26,accurate,False,20.382686,49.456130,7.791667,0.0,False,1248.576538
1,2,1001101101100271211,20,NaN,NaN,1001101101100271211/2023-09-28T06:36:05.355751...,TP1A.220624.014_2,2023-09-26 06:36:05.0,NaN,1,...,7.791667,2023-09-26,accurate,False,20.382686,49.456130,7.791667,0.0,False,1248.576538
2,3,1001101101100271311,10,NaN,NaN,1001101101100271311/2023-09-28T06:37:16.029149...,TP1A.220624.014_2,2023-09-26 06:37:16.0,NaN,0,...,7.791667,2023-09-26,accurate,False,20.382686,49.456130,7.791667,0.0,False,1248.576538
3,4,1001101101100191111,0,NaN,NaN,1001101101100191111/2023-09-28T06:51:15.117682...,TP1A.220624.014_2,2023-09-26 06:51:15.0,NaN,0,...,7.769743,2023-09-26,accurate,False,8.700529,49.448608,7.769743,0.0,False,2236.116862
4,5,1001101101100191211,10,NaN,NaN,1001101101100191211/2023-09-28T06:51:43.824174...,TP1A.220624.014_2,2023-09-26 06:51:44.0,NaN,0,...,7.769743,2023-09-26,accurate,False,8.700529,49.448608,7.769743,0.0,False,2236.116862
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
35525,61150,1001101101100531112,90,NaN,NaN,1001101101100531112/2024-10-17T14:47:10.767846...,UP1A.231005.007_0a1ae090352d8dcc,2024-10-17 14:47:11.0,NaN,1,...,7.708089,2024-10-17,NaN,NaN,NaN,49.440841,7.708089,0.0,True,1878.638672
35526,61151,1001101101100531211,20,NaN,NaN,1001101101100531211/2024-10-17T14:46:43.691739...,UP1A.231005.007_0a1ae090352d8dcc,2024-10-17 14:46:44.0,NaN,1,...,7.708089,2024-10-17,NaN,NaN,NaN,49.440841,7.708089,0.0,True,1878.638672
35527,61152,1001101101100531212,90,NaN,NaN,1001101101100531212/2024-10-17T14:46:54.523517...,UP1A.231005.007_0a1ae090352d8dcc,2024-10-17 14:46:55.0,NaN,1,...,7.708089,2024-10-17,NaN,NaN,NaN,49.440841,7.708089,0.0,True,1878.638672
35528,61153,1001101101100531311,20,NaN,NaN,1001101101100531311/2024-10-17T14:47:19.611355...,UP1A.231005.007_0a1ae090352d8dcc,2024-10-17 14:47:20.0,NaN,0,...,7.708089,2024-10-17,NaN,NaN,NaN,49.440841,7.708089,0.0,True,1878.638672


# Get Regions

In [3]:
geolocator = Nominatim(user_agent="geoapiExercises")

In [4]:
def get_city(lat, lon):
    """
    Bestimmt die nächstgelegene Stadt anhand von Latitude und Longitude.
    """
    try:
        location = geolocator.reverse((lat, lon), language="en")
        address = location.raw.get('address', {})
        return address.get('city', 'Unknown')
    except Exception as e:
        return 'Error'

In [5]:
# Städte zuordnen
df_ContainerActivities_cleaned_coordinates['CITY'] = df_ContainerActivities_cleaned_coordinates.apply(
    lambda row: get_city(row['NEW_LAT'], row['NEW_LON']), axis=1
)
df_ContainerActivities_cleaned_coordinates

KeyboardInterrupt: 

In [ ]:
# Schritt 2: Grenzen der Städte herunterladen
city_boundaries = {}
for city in df_ContainerActivities_cleaned_coordinates['CITY'].unique():
    if city != 'Unknown' and city != 'Error':
        try:
            gdf = ox.geocode_to_gdf(city)
            city_boundaries[city] = gdf
        except Exception as e:
            print(f"Fehler beim Laden der Grenzen für {city}: {e}")


In [ ]:
# Kombinierte Grenzen erstellen
if city_boundaries:
    combined_boundaries = gpd.GeoDataFrame(pd.concat(city_boundaries.values(), ignore_index=True))

    # Schritt 3: Punkte und Grenzen verknüpfen
    # Punkte in GeoDataFrame umwandeln
    points_gdf = gpd.GeoDataFrame(
        df_ContainerActivities_cleaned_coordinates,
        geometry=[Point(lon, lat) for lat, lon in zip(
            df_ContainerActivities_cleaned_coordinates['NEW_LAT'], 
            df_ContainerActivities_cleaned_coordinates['NEW_LON']
        )],
        crs="EPSG:4326"
    )

    # Zuordnung von Punkten zu Städten
    points_gdf['MATCHED_CITY'] = None
    for city, boundary in city_boundaries.items():
        points_gdf.loc[points_gdf.within(boundary.geometry.iloc[0]), 'MATCHED_CITY'] = city

    # Ergebnis anzeigen
    print(points_gdf)

    # Optional: Grenzen und Punkte visualisieren
    combined_boundaries.plot(figsize=(10, 10), color="lightblue", edgecolor="black", alpha=0.5)
    points_gdf.plot(ax=plt.gca(), color="red", markersize=50)
    plt.title("Containerpunkte und Stadtgrenzen")
    plt.show()
else:
    print("Keine Stadtgrenzen gefunden.")


# Container Density

In [4]:
# Dynamischen Radius basierend auf Bevölkerungsdichte berechnen
def calculate_dynamic_radius(pop_density, base=1000, min_radius=1, max_radius=5):
    """
    Berechnet den Radius dynamisch basierend auf der Bevölkerungsdichte.
    """
    if pop_density <= 0:
        return max_radius  # Sicherheitsmaßnahme
    radius = base / (pop_density**0.5)
    return max(min_radius, min(max_radius, radius))

In [6]:
df_ContainerActivities_cleaned_coordinates['RADIUS_KM'] = df_ContainerActivities_cleaned_coordinates['POPULATION_DENSITY'].apply(calculate_dynamic_radius)
df_ContainerActivities_cleaned_coordinates

,TRANSACTION_ID,CONTAINER_ID,SLIDER_LEVEL,CV_LEVEL,SENSOR_LEVEL,IMAGE_ID,PHONE_ID,RECORDED_AT,COMMENT,IS_EMPTIED,...,DATE,CLASSIFICATION,outlier,DISTANCE,NEW_LAT,NEW_LON,NEW_DISTANCE,INACCURATE_COORDINATES,POPULATION_DENSITY,RADIUS_KM
0,1,1001101101100271111,30,NaN,NaN,1001101101100271111/2023-09-28T06:36:48.022095...,TP1A.220624.014_2,2023-09-26 06:36:48.0,NaN,1,...,2023-09-26,accurate,False,20.382686,49.456130,7.791667,0.0,False,1248.576538,5
1,2,1001101101100271211,20,NaN,NaN,1001101101100271211/2023-09-28T06:36:05.355751...,TP1A.220624.014_2,2023-09-26 06:36:05.0,NaN,1,...,2023-09-26,accurate,False,20.382686,49.456130,7.791667,0.0,False,1248.576538,5
2,3,1001101101100271311,10,NaN,NaN,1001101101100271311/2023-09-28T06:37:16.029149...,TP1A.220624.014_2,2023-09-26 06:37:16.0,NaN,0,...,2023-09-26,accurate,False,20.382686,49.456130,7.791667,0.0,False,1248.576538,5
3,4,1001101101100191111,0,NaN,NaN,1001101101100191111/2023-09-28T06:51:15.117682...,TP1A.220624.014_2,2023-09-26 06:51:15.0,NaN,0,...,2023-09-26,accurate,False,8.700529,49.448608,7.769743,0.0,False,2236.116862,5
4,5,1001101101100191211,10,NaN,NaN,1001101101100191211/2023-09-28T06:51:43.824174...,TP1A.220624.014_2,2023-09-26 06:51:44.0,NaN,0,...,2023-09-26,accurate,False,8.700529,49.448608,7.769743,0.0,False,2236.116862,5
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
35525,61150,1001101101100531112,90,NaN,NaN,1001101101100531112/2024-10-17T14:47:10.767846...,UP1A.231005.007_0a1ae090352d8dcc,2024-10-17 14:47:11.0,NaN,1,...,2024-10-17,NaN,NaN,NaN,49.440841,7.708089,0.0,True,1878.638672,5
35526,61151,1001101101100531211,20,NaN,NaN,1001101101100531211/2024-10-17T14:46:43.691739...,UP1A.231005.007_0a1ae090352d8dcc,2024-10-17 14:46:44.0,NaN,1,...,2024-10-17,NaN,NaN,NaN,49.440841,7.708089,0.0,True,1878.638672,5
35527,61152,1001101101100531212,90,NaN,NaN,1001101101100531212/2024-10-17T14:46:54.523517...,UP1A.231005.007_0a1ae090352d8dcc,2024-10-17 14:46:55.0,NaN,1,...,2024-10-17,NaN,NaN,NaN,49.440841,7.708089,0.0,True,1878.638672,5
35528,61153,1001101101100531311,20,NaN,NaN,1001101101100531311/2024-10-17T14:47:19.611355...,UP1A.231005.007_0a1ae090352d8dcc,2024-10-17 14:47:20.0,NaN,0,...,2024-10-17,NaN,NaN,NaN,49.440841,7.708089,0.0,True,1878.638672,5


In [12]:
# Containerdichte berechnen
def calculate_density(container, df):
    """
    Berechnet die Containerdichte für einen Container, basierend auf dynamischen Radien und Regionstyp.
    """
    current_location = (container['NEW_LAT'], container['NEW_LON'])
    current_region = container['REGION_TYPE']
    current_radius = container['RADIUS_KM']
    
    # Erlaube nur Regionen gemäß Verhalten (Stadt, Land, Vorort)
    if current_region == 'Stadt':
        valid_regions = ['Stadt']
    elif current_region == 'Land':
        valid_regions = ['Land']
    elif current_region == 'Vorort':
        valid_regions = ['Vorort', 'Stadt']
    
    # Filter für Container im selben Regionstyp
    valid_containers = df[df['REGION_TYPE'].isin(valid_regions)]
    
    # Berechnung der Distanzen
    distances = valid_containers.apply(
        lambda row: geodesic(current_location, (row['NEW_LAT'], row['NEW_LON'])).km, axis=1
    )
    
    # Container im Radius zählen
    nearby_containers = distances[distances <= current_radius]
    
    # Containerdichte: Anzahl der Container im Radius pro Fläche (Pi * r^2)
    area_km2 = 3.14159 * (current_radius ** 2)
    density = len(nearby_containers) / area_km2
    
    return density

In [13]:
df_ContainerActivities_cleaned_coordinates['CONTAINER_DENSITY'] = df_ContainerActivities_cleaned_coordinates.apply(calculate_density, df=df_ContainerActivities_cleaned_coordinates, axis=1)

df_ContainerActivities_cleaned_coordinates

KeyError: 'REGION_TYPE'